In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.applications import VGG16
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framewo

In [2]:
traindf = pd.read_csv("dataset/train.csv")
testdf = pd.read_csv("dataset/test.csv")

In [3]:
traindf.head()

Image   Class
0   image7042.jpg    Food
1   image3327.jpg    misc
2  image10335.jpg  Attire
3   image8019.jpg    Food
4   image2128.jpg  Attire

In [4]:
testdf.head()

Image
0   image6245.jpg
1  image10409.jpg
2   image8692.jpg
3  image10517.jpg
4   image2580.jpg

In [5]:
len(traindf)

5983

In [6]:
len(testdf)

3219

In [7]:
traindf.groupby(by = 'Class', axis=0).count()

Image
Class                      
Attire                 1691
Decorationandsignage    743
Food                   2278
misc                   1271

In [8]:
trainPath = "dataset/Train Images/"
testPath = "dataset/Test Images/"

In [9]:
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.0808, horizontal_flip=True)

In [10]:
trainGenerator=datagen.flow_from_dataframe(dataframe=traindf,
                                           directory=trainPath,
                                           x_col="Image",
                                           y_col="Class",
                                           batch_size=50,
                                           subset="training",
                                           seed=42,
                                           shuffle=True,
                                           class_mode="categorical",
                                           target_size=(60,80))

validGenerator=datagen.flow_from_dataframe(dataframe=traindf,
                                           directory=trainPath,
                                           x_col="Image",
                                           y_col="Class",
                                           batch_size=23,
                                           subset="validation",
                                           seed=42,
                                           shuffle=True,
                                           class_mode="categorical",
                                           target_size=(60,80))

Found 5500 validated image filenames belonging to 4 classes.
Found 483 validated image filenames belonging to 4 classes.


In [11]:
testDatagen=ImageDataGenerator(rescale=1./255.)

testGenerator=testDatagen.flow_from_dataframe(dataframe=testdf,
                                                directory=testPath,
                                                x_col="Image",
                                                y_col=None,
                                                batch_size=29,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=(60,80))

Found 3219 validated image filenames.


In [12]:
def buildModel():
    baseModel = VGG16(input_shape=(60, 80, 3), weights='imagenet',include_top=False)
    model = Sequential()
    model.add(baseModel)
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))
    
    baseModel.trainable=False
    
    set_trainable = False
    for layer in baseModel.layers:
        if layer.name == 'layer_name':
            set_trainable = True
        
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return baseModel, model

In [13]:
baseModel, model = buildModel()

baseModel.summary()
model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60, 80, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 60, 80, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 60, 80, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 30, 40, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 30, 40, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 30, 40, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 15, 20, 128)       0    

In [14]:
STEP_SIZE_TRAIN=trainGenerator.n//trainGenerator.batch_size
STEP_SIZE_VALID=validGenerator.n//validGenerator.batch_size

checkpointer = ModelCheckpoint(filepath='weights/model.ckpt', verbose=1, save_best_only=True)

In [15]:
model.fit_generator(generator=trainGenerator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=validGenerator, validation_steps=STEP_SIZE_VALID, epochs=50, verbose=1, callbacks=[checkpointer])


Epoch 1/50
110/110 [==============================] - 123s 1s/step - loss: 1.1313 - accuracy: 0.5064 - val_loss: 1.2694 - val_accuracy: 0.5424

Epoch 00001: val_loss improved from inf to 1.26943, saving model to weights/model.ckpt
Epoch 2/50
110/110 [==============================] - 124s 1s/step - loss: 0.9680 - accuracy: 0.5951 - val_loss: 1.0900 - val_accuracy: 0.5652

Epoch 00002: val_loss improved from 1.26943 to 1.09000, saving model to weights/model.ckpt
Epoch 3/50
110/110 [==============================] - 125s 1s/step - loss: 0.9185 - accuracy: 0.6240 - val_loss: 0.8169 - val_accuracy: 0.5590

Epoch 00003: val_loss improved from 1.09000 to 0.81694, saving model to weights/model.ckpt
Epoch 4/50
110/110 [==============================] - 129s 1s/step - loss: 0.8714 - accuracy: 0.6467 - val_loss: 0.9053 - val_accuracy: 0.5797

Epoch 00004: val_loss did not improve from 0.81694
Epoch 5/50
110/110 [==============================] - 140s 1s/step - loss: 0.8472 - accuracy: 0.6509 - 


Epoch 00040: val_loss did not improve from 0.55278
Epoch 41/50
110/110 [==============================] - 163s 1s/step - loss: 0.2175 - accuracy: 0.9271 - val_loss: 1.8614 - val_accuracy: 0.5942

Epoch 00041: val_loss did not improve from 0.55278
Epoch 42/50
110/110 [==============================] - 162s 1s/step - loss: 0.1935 - accuracy: 0.9436 - val_loss: 1.3546 - val_accuracy: 0.5963

Epoch 00042: val_loss did not improve from 0.55278
Epoch 43/50
110/110 [==============================] - 162s 1s/step - loss: 0.1942 - accuracy: 0.9400 - val_loss: 0.5316 - val_accuracy: 0.6046

Epoch 00043: val_loss improved from 0.55278 to 0.53157, saving model to weights/model.ckpt
Epoch 44/50
110/110 [==============================] - 162s 1s/step - loss: 0.1883 - accuracy: 0.9411 - val_loss: 1.6561 - val_accuracy: 0.5735

Epoch 00044: val_loss did not improve from 0.53157
Epoch 45/50
110/110 [==============================] - 163s 1s/step - loss: 0.1885 - accuracy: 0.9378 - val_loss: 1.1965 - v

In [16]:
model.evaluate_generator(generator=validGenerator, steps=STEP_SIZE_VALID, verbose=1)

21/21 [==============================] - 10s 494ms/step


[0.7667936086654663, 0.5714285969734192]

In [17]:
model.load_weights('weights/model.ckpt')

In [18]:
model.evaluate_generator(generator=validGenerator, steps=STEP_SIZE_VALID, verbose=1)

21/21 [==============================] - 10s 489ms/step


[1.3189927339553833, 0.5879917144775391]

In [19]:
model.save_weights('weights/model.h5')

In [20]:
STEP_SIZE_TEST=testGenerator.n//testGenerator.batch_size
pred = model.predict_generator(testGenerator, steps=STEP_SIZE_TEST, verbose=1)
pred

111/111 [==============================] - 93s 835ms/step


array([[7.3502627e-03, 3.8988065e-02, 9.4088435e-01, 1.2777319e-02],
       [1.2494288e-01, 2.4364052e-02, 5.0449836e-01, 3.4619471e-01],
       [1.0984734e-04, 3.3915512e-06, 9.9906760e-01, 8.1914070e-04],
       ...,
       [6.2582636e-01, 2.7519016e-05, 3.7473146e-05, 3.7410861e-01],
       [4.0103358e-05, 4.5134325e-02, 9.8777306e-04, 9.5383787e-01],
       [5.6510672e-02, 1.0006525e-02, 4.3034058e-02, 8.9044875e-01]],
      dtype=float32)

In [21]:
pred.shape

(3219, 4)

In [22]:
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices

array([2, 2, 2, ..., 0, 3, 3])

In [23]:
predicted_class_indices.shape

(3219,)

In [24]:
labels = (trainGenerator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions

['Food',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'misc',
 'misc',
 'Attire',
 'Decorationandsignage',
 'Decorationandsignage',
 'misc',
 'Food',
 'Food',
 'Food',
 'Attire',
 'misc',
 'misc',
 'Attire',
 'Food',
 'misc',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'misc',
 'Attire',
 'Attire',
 'Food',
 'Decorationandsignage',
 'Decorationandsignage',
 'misc',
 'Attire',
 'Food',
 'Food',
 'Decorationandsignage',
 'misc',
 'misc',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'misc',
 'Food',
 'Food',
 'Attire',
 'misc',
 'Decorationandsignage',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Decorationandsignage',
 'misc',
 'Attire',
 'Food',
 'Decorationandsignage',
 'Food',
 'Food',
 'misc',
 'Attire',
 'misc',
 'Food',
 'Food',
 'Decorationandsignage',
 'Food',
 'Decorationandsignage',
 'Attire',
 'Food',
 'misc',
 'Attire',
 'Attire',
 'Food',
 'misc',
 'Food',
 'Food',
 'Food',
 'Decorationandsignage',
 'Atti

In [25]:
filenames=testGenerator.filenames
results=pd.DataFrame({"Image":filenames, "Class":predictions})
results

Image                 Class
0      image6245.jpg                  Food
1     image10409.jpg                  Food
2      image8692.jpg                  Food
3     image10517.jpg                  Food
4      image2580.jpg                Attire
...              ...                   ...
3214   image4968.jpg  Decorationandsignage
3215   image6673.jpg                  Food
3216   image3442.jpg                Attire
3217   image8178.jpg                  misc
3218   image5367.jpg                  misc

[3219 rows x 2 columns]

In [27]:
results.groupby(by = 'Class', axis=0).count()

Image
Class                      
Attire                  856
Decorationandsignage    474
Food                   1206
misc                    683

In [26]:
results.to_csv("output/output3.csv", index=False)